In [1]:
!pip install PyPDF2

     |████████████████████████████████| 77 kB 3.0 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=a2de9452e14145c2f072412494512f71595a9d3f823802204005ca1070311bf6
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [2]:
from urllib import request

def download_file(download_url,name):
    request.urlretrieve(download_url, f'{name}.pdf')

In [3]:
import PyPDF2

In [4]:
def pdf2str(pdf_link):
    download_file(pdf_link ,'sentenza')
    pdfFileObj = open('sentenza.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    text = ''
    for p in range(pdfReader.numPages):
        text +=  pdfReader.getPage(p).extractText()
    !rm 'sentenza.pdf'
    return text

In [5]:
from urllib.parse import unquote

url = unquote('http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211109/snpen@s20@a2021@n40338@tS.clean.pdf')

In [6]:
pdf2str('http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211109/snpen@s20@a2021@n40338@tS.clean.pdf')

'SENTENZA sui ricorsi proposti da: BELFIORE TOMMASO EMANUELE CINCINNATO MARCELLO EPICOCO ANTONIO GUARINI EMANUELE avverso la sentenza del 26/09/2019 nato a MESAGNE il 24/09/1968 nato a MESAGNE il 30/11/1967 nato a BRINDISI il 23/06/1974 nato a MESAGNE il 14/06/1974 della CORTE DI ASSISE DI APPELLO DI TARANTO visti gli atti, il provvedimento impugnato e i ricorsi; udita la relazione svolta dal Consigliere Piero MESSINI D\'AGOSTINI; udito il Pubblico Ministero, in persona del Sostituto Procuratore generale Fulvio BALDI, che ha concluso per l\'annullamento con rinvio riguardo alle posizioni di Cincinnato, Epicoco e Guarini e per l\'inammissibilità del ricorso di Belfiore; 1 Penale Sent. Sez. 2   Num. 40338  Anno 2021Presidente: DIOTALLEVI GIOVANNIRelatore: MESSINI D\'AGOSTINI PIEROData Udienza: 19/10/2021udito il difensore della parte civile Comune di Mesagne, avv. Domenico DI SIENA, che ha concluso per la conferma della sentenza; uditi i difensori degli imputati, avv. Angela M. Rosaria E

In [7]:
!pip install bs4

In [8]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=947ac015135561b61ee8f1ce14f41d07720d6fc78cb571f59577cc29043da08d
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [9]:
# Basic libraries
import pandas as pd
import numpy as np
import nltk

# Importing for scraping
from bs4 import BeautifulSoup 
import fake_useragent


import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation

In [10]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 958 kB 4.9 MB/s 
     |████████████████████████████████| 356 kB 48.3 MB/s 
     |████████████████████████████████| 138 kB 52.9 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 3.6 MB 41.6 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
from selenium.webdriver.support.ui import Select

In [11]:
driver.get('http://www.italgiure.giustizia.it/sncass/')

In [12]:
driver.find_element_by_id('1.[kind]').click()

In [14]:
driver.find_elements_by_class_name('card')

[<selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="515eaaa4-1137-4d9f-9670-16b447f9d319")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="998db92f-c8be-41ea-8a45-a9f45dc53a70")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="4257bf67-9839-4be9-8e8f-67e6daadcef0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="e931d5ff-7c85-4509-bac2-c79080ef84bb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="2146f539-8389-41cd-a78b-c62a321099a2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="d9486e81-db15-4e65-8df1-d69e7becfdca")>,
 <selenium.webdriver.remote.webelement.WebElement (session="81a2c7986033e523db512e14a7cac016", element="07a6906e-32bc-49ca-ae5f-6b

In [18]:
oscurato = driver.find_elements_by_class_name('card')[-1]

visibie = driver.find_elements_by_class_name('card')[-2]

In [34]:
link1 = visibie.find_elements_by_css_selector('a')[1].find_elements_by_css_selector('span')[0].find_element_by_css_selector('span').get_attribute('data-arg')

In [35]:
link2 = oscurato.find_elements_by_css_selector('a')[1].find_elements_by_css_selector('span')[0].find_element_by_css_selector('span').get_attribute('data-arg')

In [36]:
link2

'/xway/application/nif/clean/hc.dll%3Fverbo%3Dattach%26db%3Dsnciv%26id%3D./20211220/snciv@s61@a2021@n40903@tO@oY.clean.pdf'

In [37]:
from urllib.parse import unquote

url1 = unquote(link1)
url2 = unquote(link2)

In [40]:
print('http://www.italgiure.giustizia.it' + url1 )
print('http://www.italgiure.giustizia.it' + url2 )

http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snciv&id=./20211220/snciv@sL0@a2021@n40904@tO.clean.pdf
http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snciv&id=./20211220/snciv@s61@a2021@n40903@tO@oY.clean.pdf
